In [1]:
from pelux_core import *
import os

Could not import regions, which is required for some of the functionalities of this module.


# Catalogue queries

The first step of the program consists in collecting data about a given list of stars.

You can try searching for coordinates and photometry of some stars:

In [2]:
path=r'C:\Users\Vito\Desktop\PhD\Programmi\Python\prove'
my_file=os.path.join(path,'prova.txt')

search_phot(my_file)

C:\Users\Vito\AppData\Local\Programs\Python\Python38\lib\site-packages\astroquery\simbad\core.py:135: UserWarning: Warning: The script line number 3 raised an error (recorded in the `errors` attribute of the result table): 'Fake Star': No known catalog could be found
  warnings.warn("Warning: The script line number %i raised "


Star Fake_Star  not found. Perhaps misspelling? Setting (ra,dec) to NaN.
Some stars were not found. Would you like to end the program and check the spelling?
If not, these stars will be treated as missing data
End program? [Y/N]n


C:\Users\Vito\AppData\Local\Programs\Python\Python38\lib\site-packages\tabulate.py:595: UserWarning: Warning: converting a masked element to nan.
  conv(string)
C:\Users\Vito\AppData\Local\Programs\Python\Python38\lib\site-packages\tabulate.py:996: UserWarning: Warning: converting a masked element to nan.
  return format(float(val), floatfmt)


The file can be either a list of equatorial coordinates (RA, dec), with header 'ra_v, dec_v' (.txt or .csv allowed), or a list of star names (no header, only .txt allowed). In the former case, the keyword coordinates must be set to 'True'.

A meaningful name for the input file is suggested, as it will be the prefix for all the files creating during the execution. It should also be distinguishable from files already present in the working path to avoid overwriting.

If one or more star names do not have a match in Simbad, the program prints their names, then asks you the permission to continue. Typing "Yes", it will fill the row(s) with NaN, resulting in a Null result for the related entry. If instead you wish to edit the file (e.g. due to a misspelling), type "No".

In default mode, the employed catalogues are the following:
- Gaia EDR3;
- 2MASS.

To specify a different set of catalogues, you can use the keyword surveys:

In [3]:
coord_file=os.path.join(path,'prova_coordinates.csv')
search_phot(coord_file,coordinates=True,surveys=['GAIA_EDR3','ALLWISE'])

Available surveys (19/04): 'Gaia_EDR3', '2MASS', 'ALLWISE'. **Vanno aggiunti almeno Gaia DR2, WISE e Panstarrs**

In the working path you can see a file ending with '\_coordinates.csv'. It is directly related to the input file (being a mere copy of it if it was a coordinate file) and will constitute the reference file for internal IDs within the sample. Moreover, there's a file named after each survey ('\_data'), and containing the collected data.

The module load_phot cross-matches these data in single photometric matrix [n,m] with one row per star and one column per filter*. A similar phot_err matrix, containing the photometric errors, is returned too, together with an m-elements list specifying the filters corresponding to every column.

\*The internal function cross_match takes (ra, dec) of the reference file and of the i-th surveys, and identifies a source as being the same if the |d(RA)|+|d(dec)|<3.6'' (**Dovrei renderlo un parametro modificabile a piacimento**). If it founds more than one source, it picks the brightest one.

In [4]:
phot,phot_err,filters,kin=load_phot(coord_file,['GAIA_EDR3','2MASS','ALLWISE'])
print(filters)

['G', 'GBP', 'GRP', 'J', 'H', 'K', 'W1', 'W2', 'W3', 'W4']


All the photometry has been saved in the file 'all_photometry.txt'.

# Preparation of data

The module load_phot creates also a file named '\_kinematics.txt' and containing Gaia positions, parallaxes and proper motions.
(**Dovrei recuperare queste informazioni da Gaia DR2 o Hipparcos, se non ci sono in Gaia EDR3**)

With parallaxes, we can compute absolute magnitudes:


In [5]:
par=kin[:,4]
par_err=kin[:,5]
abs_phot,abs_err=app_to_abs_mag(phot,par,app_mag_error=phot_err,parallax_error=par_err)
print('The star 1 with apparent magnitudes\n',phot[0,:],'\n in the filters',filters,'\nand parallax ',par[0],'mas \nhas absolute magnitudes\n',abs_phot[0,:])

The star 1 with apparent magnitudes
 [4.9298 4.8706 5.028  5.204  5.292  5.314  5.359  5.197  5.446  5.388 ] 
 in the filters ['G', 'GBP', 'GRP', 'J', 'H', 'K', 'W1', 'W2', 'W3', 'W4'] 
and parallax  7.7553 mas 
has absolute magnitudes
 [-0.62220699 -0.68140699 -0.52400699 -0.34800699 -0.26000699 -0.23800699
 -0.19300699 -0.35500699 -0.10600699 -0.16400699]


**Va integrata la parte di estinzione e quella di contaminazione, ancora da scrivere**

The extinction in the position given by ($\alpha$, $\delta$, $\pi$) can be computed via the module interstellar_ext, based on the galactic 3D extinction map by Leike et al. (2020):

**Va aggiunta la mappa di Lallement et al. 2019**


In [7]:
ext=interstellar_ext(ra=kin[:,0],dec=kin[:,2],par=par,ext_map='leike',color='G')
print('The star 1 with ra=',kin[0,0],', dec=',kin[0,2],' and parallax ',par[0],'mas \nhas G-band extinction',ext[0],'\n according to Leike et al. 2020')

The star 1 with ra= 155.7420251189 , dec= -66.90144452703  and parallax  7.7553 mas 
has G-band extinction 0.08139007905034745 
 according to Leike et al. 2020
